# 4 - Deployment Phase

In [1]:
# Author information
__author__ = "Troy Reynolds"
__email__ = "Troy.Lloyd.Reynolds@gmail.com"

In [2]:
## Libraries
# standard packages
import pandas as pd
import sys

# Add data_storage and helper_functions to directory
sys.path.insert(0, "./data_storage")
sys.path.insert(0, "./helper_functions")
sys.path.insert(0, "./model")

# import helper functions
from file_management import data_import
from deployment import deployment_pipeline, test_data_metrics

# Load Test Data

In [3]:
# load in data
test_data = data_import("test")
features = test_data.drop("Exited", axis = 1)
labels = test_data["Exited"]
test_data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasChckng,IsActiveMember,EstimatedSalary,Exited
5702,5703,15749540,Hsiung,585.0,West,Male,36,7,0.00,2,1,0,94283.09,0
3667,3668,15807340,O'Donnell,525.0,East,Male,33,4,131023.76,2,0,0,55072.93,0
1617,1618,15801062,Matthews,557.0,Central,Female,40,4,0.00,2,0,1,105433.53,0
5673,5674,15572801,Krischock,639.0,Central,Male,34,5,139393.19,2,0,0,33950.08,0
4272,4273,15600708,Calabresi,640.0,Central,Female,34,3,77826.80,1,1,1,168544.85,0


# Run test features through prediction pipeline
The test features are processed using the pipeline created in the previous notebook. The processed features are run through the best model to get predictions based on the optimal threshold.

In [4]:
#### Get predictions
predictions = deployment_pipeline(features, threshold = 0.2)
predictions[:10]

[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]

# Results on test data

In [5]:
test_data_metrics(labels, predictions)


****************************** Test Data Metrics *******************************
Accuracy:	0.779
Precision:	0.4740740740740741
Recall:		0.7862407862407862


# GUI Prototyping

In [6]:
# enable widgest
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [7]:
### Libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

### Background Operations and functions

#### Get data

In [8]:
train_data = data_import("train")

#### Widget Creation

In [9]:
### TextBoxes
input_CreditScore = widgets.BoundedIntText(
    value=650,
    min=train_data.CreditScore.min(),
    max=train_data.CreditScore.max(),
    step=1,
    description='Credit Score:',
    disabled=False
)
input_Balance = widgets.BoundedIntText(
    value=0,
    min=train_data.Balance.min(),
    max=train_data.Balance.max(),
    step=1,
    description='Balance:',
    disabled=False
)
input_EstimatedSalary = widgets.BoundedIntText(
    value=0,
    min=train_data.EstimatedSalary.min(),
    max=train_data.EstimatedSalary.max(),
    step=1,
    description='Salary:',
    disabled=False
)

### Sliders
input_age = widgets.IntSlider(
    min = train_data.Age.min(),
    max = train_data.Age.max(),
    step = 1,
    description = "Age: ",
    value = 50
)
input_tenure = widgets.IntSlider(
    min = train_data.Tenure.min(),
    max = train_data.Tenure.max(),
    step = 1,
    description = "Tenure: ",
    value = 50
)

### Dropdowns
# function to get dropdowns
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

# widgets
input_geography = widgets.Dropdown(
    options = unique_sorted_values(train_data.Geography),
    description = 'Location:'
)
input_gender = widgets.Dropdown(
    options = unique_sorted_values(train_data.Gender),
    description = 'Gender:'
)
input_numproducts = widgets.Dropdown(
    options = unique_sorted_values(train_data.NumOfProducts),
    description = '# Products:'
)

### Checkboxes
input_hasChecking = widgets.Checkbox(
    value=False,
    description='Has Checking',
    disabled=False,
    indent=True
)

input_Active = widgets.Checkbox(
    value=False,
    description='Active User',
    disabled=False,
    indent=True
)

In [10]:
### Horizontal stacking widgets
input_textbox = widgets.HBox([input_CreditScore, input_Balance, input_EstimatedSalary])
input_slider = widgets.HBox([input_age, input_tenure])
input_dropdown = widgets.HBox([input_geography, input_gender, input_numproducts])
input_checkbox = widgets.HBox([input_hasChecking, input_Active])

### Vertical stacking
dashboard = widgets.VBox([input_textbox, input_slider, input_dropdown, input_checkbox])

#### Collect input data

In [11]:
# create button
button = widgets.Button(description='Predict!')
out = widgets.Output()

def btn_event_handler(obj):
    with out:
        clear_output()
        
        # gather data
        input_data = pd.DataFrame({
            "RowNumber": 1,
            "CustomerId": 1,
            "Surname": "Doe", #id
            "CreditScore": float(input_CreditScore.value),
            "Balance": float(input_Balance.value),
            "EstimatedSalary": float(input_EstimatedSalary.value),
            "Age": float(input_age.value),
            "Tenure": float(input_tenure.value),
            "Geography": input_geography.value,
            "Gender": input_gender.value,
            "NumOfProducts": int(input_numproducts.value),
            "HasChckng": int(input_hasChecking.value),
            "IsActiveMember": int(input_Active.value)}, index = [1])
        
        # make prediction
        prediction = deployment_pipeline(input_data, threshold = 0.2)
        
        # output
        if prediction[0] == 1:
            print("\nThe client is likely to churn.")
        
        else:
            print("\nThe client is unlikely to churn.")
    
button.on_click(btn_event_handler)

### Data Input Dashboard

In [12]:
display(dashboard)

#### Prediction of new client

In [13]:
display(widgets.VBox([button, out]))